In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
from sklearn.metrics import r2_score
import numpy as np
from torch.cuda.amp import GradScaler, autocast
from torchvision.transforms import functional as F
import pickle
import multiprocessing as mp
#import cv2
import numpy as np
#from skimage import color
#from torchmetrics.regression import R2Score


from torchvision.models import ResNet18_Weights, ResNet50_Weights

In [ ]:
os.chdir('/home/jovyan')
current_dir = os.getcwd()
print("Current working directory:", current_dir)

In [28]:
# Navigate back to the root directory -- do it 2 times
os.chdir('..')
os.chdir('..')

# Get the current working directory
current_dir = os.getcwd()
print("Current working directory:", current_dir)


Current working directory: /


In [3]:
import os

merged_posts_data = pd.read_csv('filtered_mapped_posts_2k_per_inflCateg.csv')

merged_posts_data_no_duplicates = merged_posts_data.drop_duplicates()


# Construct the image file name and check if it exists in the directory
merged_posts_data_no_duplicates['constructed_file_name'] = merged_posts_data_no_duplicates['OwnerUsername'] + '-' + merged_posts_data_no_duplicates['Image_file_name'] 

# Change to the directory containing the images
os.chdir('images_output/output_images')

# List of available image files in the directory
available_images = set(os.listdir('.'))


# Navigate back to the root directory -- do it 2 times
os.chdir('..')
os.chdir('..')

# Get the current working directory
current_dir = os.getcwd()
print("Current working directory:", current_dir)


# Filter the DataFrame to only include rows with existing images
filtered_df = merged_posts_data_no_duplicates[merged_posts_data_no_duplicates['constructed_file_name'].isin(available_images)]

# Reset the index of the filtered DataFrame
filtered_df.reset_index(drop=True, inplace=True)

# Apply label encoding to the 'Category' column
filtered_df['CategoryEncoded'] = pd.factorize(filtered_df['Category'])[0]

metadata_columns = ['PostID', 'ImageWidth', 'ImageHeight', 'CategoryEncoded', 'OwnerID', 'CommentsCount', '#Followers', '#Followees', '#Posts']
filtered_df = filtered_df[['constructed_file_name'] + metadata_columns + ['LikesNum']]


    


/tmp/ipykernel_16786/1682639279.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_posts_data_no_duplicates['constructed_file_name'] = merged_posts_data_no_duplicates['OwnerUsername'] + '-' + merged_posts_data_no_duplicates['Image_file_name']


Current working directory: /home/jovyan


/tmp/ipykernel_16786/1682639279.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['CategoryEncoded'] = pd.factorize(filtered_df['Category'])[0]


In [3]:
images = ['jordanhefler-1896533999079959560.jpg',
 'jordanhefler-1896534007569376702.jpg',
 'jordanhefler-1896534017811791152.jpg',
 'jordanhefler-1896534029799223578.jpg',
 'jordanhefler-1896534034622533248.jpg',
 'jordanhefler-1896534043640325572.jpg',
 'jordanhefler-1896534051659720341.jpg',
 'jordanhefler-1896534060644024200.jpg',
 'jordanhefler-1896534062212763393.jpg',
 'jordanhefler-1896534106504538166.jpg']

In [4]:
image_dir = 'images_output/output_images'
batch_size = 128
num_epochs = 20

visual_input_size = 2048  # Output size of ResNet50

  

img_paths = [os.path.join(image_dir, filtered_df.iloc[idx]['constructed_file_name']) for idx in range(len(filtered_df))]
  
 
#img_paths = [filtered_df.iloc[idx]['constructed_file_name'] for idx in range(len(filtered_df))]
  
 

img_paths


['images_output/output_images/0ldisme-1638622741936691543.jpg',
 'images_output/output_images/0ldisme-1639397040662477628.jpg',
 'images_output/output_images/0ldisme-1641477353219752634.jpg',
 'images_output/output_images/0ldisme-1642217994434222637.jpg',
 'images_output/output_images/0ldisme-1643019928044937006.jpg',
 'images_output/output_images/0ldisme-1643649693990809517.jpg',
 'images_output/output_images/0ldisme-1644416338749530727.jpg',
 'images_output/output_images/0ldisme-1645786010388487208.jpg',
 'images_output/output_images/0ldisme-1646568515076926791.jpg',
 'images_output/output_images/0ldisme-1647325677554422496.jpg',
 'images_output/output_images/0ldisme-1648070177016861887.jpg',
 'images_output/output_images/0ldisme-1648774302889396359.jpg',
 'images_output/output_images/0ldisme-1649471700972080657.jpg',
 'images_output/output_images/0ldisme-1650190374762769320.jpg',
 'images_output/output_images/0ldisme-1650797388186778245.jpg',
 'images_output/output_images/0ldisme-16

In [5]:
image_dir = 'images_output/output_images'

# Generate full paths for images you want to find
found_images = [os.path.join(image_dir, img) for img in images if os.path.exists(os.path.join(image_dir, img))]

# Output the paths of found images
if found_images:
    print("Found image paths:")
    for path in found_images:
        print(path)
else:
    print("No images found in the specified directory.")

Found image paths:
images_output/output_images/jordanhefler-1896533999079959560.jpg
images_output/output_images/jordanhefler-1896534007569376702.jpg
images_output/output_images/jordanhefler-1896534017811791152.jpg
images_output/output_images/jordanhefler-1896534029799223578.jpg
images_output/output_images/jordanhefler-1896534034622533248.jpg
images_output/output_images/jordanhefler-1896534043640325572.jpg
images_output/output_images/jordanhefler-1896534051659720341.jpg
images_output/output_images/jordanhefler-1896534060644024200.jpg
images_output/output_images/jordanhefler-1896534062212763393.jpg
images_output/output_images/jordanhefler-1896534106504538166.jpg


In [33]:
# Define Image Feature Extractor ResNet
class ImageFeatureExtractor(nn.Module):
    def __init__(self):
        super(ImageFeatureExtractor, self).__init__()
        self.resnet = models.resnet50(weights=ResNet50_Weights.DEFAULT)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])  # Remove the last fully connected layer

    def forward(self, x):
        with torch.no_grad():  # We do not train this part
            x = self.resnet(x)
        x = torch.flatten(x, 1)
        return x

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

class ImageFeatureExtractor_VGG19(nn.Module):
    def __init__(self):
        super(ImageFeatureExtractor_VGG19, self).__init__()
        # Load the VGG19 model pre-trained on ImageNet
        self.vgg19 = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
        # Use only up to the layer before the final classification layer
        self.features = nn.Sequential(*list(self.vgg19.children())[:-2])
        # Adding the fully connected layers, keeping up to fc7
        self.fc7 = self.vgg19.classifier[:-1]

    def forward(self, x):
        with torch.no_grad():  # We do not train this part
            x = self.features(x)
            x = torch.flatten(x, 1)  # Flatten before passing through FC layers
            x = self.fc7(x)  # Get the 4096-dimensional feature vector
        return x
        

In [5]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision.models import vit_b_16, ViT_B_16_Weights

# Define Image Feature Extractor using Vision Transformer (ViT)
class ImageFeatureExtractor_ViT(nn.Module):
    def __init__(self):
        super(ImageFeatureExtractor_ViT, self).__init__()
        # Load the Vision Transformer (ViT-B/16) model pre-trained on ImageNet
        self.vit = vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_V1)
        # Remove the classification head
        self.vit.heads = nn.Identity()

    def forward(self, x):
        with torch.no_grad():  # We do not train this part
            x = self.vit(x)
        return x


In [6]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
from torchvision import transforms
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Define a custom Dataset class for loading images
class ImageDataset(Dataset):
    def __init__(self, img_paths, transform=None):
        self.img_paths = img_paths
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_path

# Directory to save feature vectors
feature_vector_dir = 'saved_feature_vectors_VisTrasformer_2'
os.makedirs(feature_vector_dir, exist_ok=True)

# Set device and initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_model = ImageFeatureExtractor_ViT().to(device)  # ImageFeatureExtractor().to(device)
image_model.eval()

# Transformations
image_dataset_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to process a batch of images and save their feature vectors
def process_batch(batch):
    images, img_paths = batch
    images = images.to(device)
    with torch.no_grad():
        feature_vectors = image_model(images).cpu().numpy()  # Get feature vectors
    for feature_vector, img_path in zip(feature_vectors, img_paths):
        image_name = os.path.basename(img_path)
        save_path = os.path.join(feature_vector_dir, f"{image_name}.npy")
        np.save(save_path, feature_vector)

image_dataset = ImageDataset(img_paths, transform=image_dataset_transform)
data_loader = DataLoader(image_dataset, batch_size=256, shuffle=False, num_workers=16)

# Process images in batches
for batch in tqdm(data_loader, total=len(data_loader)):
    process_batch(batch)

print("Feature extraction and saving completed.")


 95%|█████████▌| 6240/6566 [4:21:34<06:13,  1.15s/it]   IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
from torchvision import transforms
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Directory to save feature vectors
feature_vector_dir = 'saved_feature_vectors_VisTrasformer_1'
os.makedirs(feature_vector_dir, exist_ok=True)

# Set device and initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_model = ImageFeatureExtractor_ViT().to(device)  # ImageFeatureExtractor().to(device)
image_model.eval()

# Transformations
image_dataset_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to process a single image and save its feature vector
def process_image(image_path):
    try:
        image = Image.open(image_path).convert('RGB')
        input_tensor = image_dataset_transform(image).unsqueeze(0).to(device)  # Add batch dimension

        # Forward pass through the model
        with torch.no_grad():
            feature_vector = image_model(input_tensor).squeeze().cpu().numpy()  # 2048-dimensional

        # Save the feature vector
        image_name = os.path.basename(image_path)
        save_path = os.path.join(feature_vector_dir, f"{image_name}.npy")
        np.save(save_path, feature_vector)
        #return f"Feature vector extracted for {image_name}"
    except Exception as e:
        return f"Error processing {image_path}: {e}"


# Use ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor(max_workers=8) as executor:  # Adjust max_workers based on your CPU/GPU resources
    results = list(tqdm(executor.map(process_image, img_paths), total=len(img_paths)))

# Print results
#for result in results:
#print(result)


 39%|███▉      | 661280/1680655 [6:24:11<26:49:13, 10.56it/s] 

In [ ]:
import os
import numpy as np

# Directory to save feature vectors
feature_vector_dir = 'saved_feature_vectors_VisTrasformer'
os.makedirs(feature_vector_dir, exist_ok=True)



# Set device and initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_model = ImageFeatureExtractor_ViT().to(device) #ImageFeatureExtractor().to(device)
image_model.eval()

  
image_dataset_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        #transforms.RandomHorizontalFlip(),
        #transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



# Extract feature vectors
feature_vectors = {}
for image_path in img_paths: #found_images:
    try:
        image = Image.open(image_path).convert('RGB')
        input_tensor = image_dataset_transform(image).unsqueeze(0).to(device)  # Add batch dimension

        # Forward pass through ResNet-50
        with torch.no_grad():
            feature_vector = image_model(input_tensor).squeeze().cpu().numpy()  # 2048-dimensional

        # Store the feature vector
        image_name = os.path.basename(image_path)
        feature_vectors[os.path.basename(image_path)] = feature_vector
        save_path = os.path.join(feature_vector_dir, f"{image_name}.npy")
        np.save(save_path, feature_vector)
        #print(f"Feature vector extracted for {image_path}")

    except Exception as e:
        print(f"Error processing {image_path}: {e}")

# Output results
#print("Feature vectors extracted:")
#for image_name, vector in feature_vectors.items():
#print(f"{image_name}: {vector.shape}")

In [13]:
from scipy.spatial.distance import cosine
import numpy as np
import pandas as pd


# List of image names for reference
image_names = list(feature_vectors.keys())


# List to store pairwise similarities
pairwise_similarities = []

# Calculate pairwise cosine similarity for each unique pair of images
for i, img1 in enumerate(image_names):
    for j in range(i + 1, len(image_names)):  # Only calculate for j > i to avoid duplicate pairs
        img2 = image_names[j]
        # Cosine similarity: 1 - cosine distance
        similarity = 1 - cosine(feature_vectors[img1], feature_vectors[img2])
        pairwise_similarities.append((img1, img2, similarity))

# Sort the list of pairs by similarity in descending order
pairwise_similarities.sort(key=lambda x: x[2], reverse=True)

# Display sorted pairwise similarities
print("Pairwise Cosine Similarities (sorted):")
for img1, img2, similarity in pairwise_similarities:
    print(f"{img1} - {img2}: {similarity:.4f}")



# Initialize a list to store all pairwise similarity values (excluding self-similarities)
similarities = []

# Calculate pairwise cosine similarity for each unique pair of images
for i, img1 in enumerate(image_names):
    for j in range(i + 1, len(image_names)):  # Only calculate for j > i to avoid self-similarities
        img2 = image_names[j]
        # Cosine similarity: 1 - cosine distance
        similarity = 1 - cosine(feature_vectors[img1], feature_vectors[img2])
        similarities.append(similarity)

# Calculate the average similarity
average_similarity = sum(similarities) / len(similarities)

print(f"Average Cosine Similarity (excluding self-similarity): {average_similarity:.4f}")


Pairwise Cosine Similarities (sorted):
jordanhefler-1896533999079959560.jpg - jordanhefler-1896534034622533248.jpg: 0.6044
jordanhefler-1896534034622533248.jpg - jordanhefler-1896534106504538166.jpg: 0.4479
jordanhefler-1896533999079959560.jpg - jordanhefler-1896534106504538166.jpg: 0.4302
jordanhefler-1896534017811791152.jpg - jordanhefler-1896534034622533248.jpg: 0.4029
jordanhefler-1896533999079959560.jpg - jordanhefler-1896534017811791152.jpg: 0.3791
jordanhefler-1896534017811791152.jpg - jordanhefler-1896534106504538166.jpg: 0.3643
jordanhefler-1896534007569376702.jpg - jordanhefler-1896534062212763393.jpg: 0.3277
jordanhefler-1896534043640325572.jpg - jordanhefler-1896534106504538166.jpg: 0.2419
jordanhefler-1896534017811791152.jpg - jordanhefler-1896534043640325572.jpg: 0.2313
jordanhefler-1896534034622533248.jpg - jordanhefler-1896534043640325572.jpg: 0.1719
jordanhefler-1896534007569376702.jpg - jordanhefler-1896534051659720341.jpg: 0.1373
jordanhefler-1896534029799223578.jpg 

In [8]:
# Dictionary to hold loaded feature vectors
loaded_feature_vectors = {}
feature_vector_dir = 'saved_feature_vectors_VisTrasformer_2'


images = ['jordanhefler-1896533999079959560.jpg',
 'jordanhefler-1896534007569376702.jpg',
 'jordanhefler-1896534017811791152.jpg',
 'jordanhefler-1896534029799223578.jpg',
 'jordanhefler-1896534034622533248.jpg',
 'jordanhefler-1896534043640325572.jpg',
 'jordanhefler-1896534051659720341.jpg',
 'jordanhefler-1896534060644024200.jpg',
 'jordanhefler-1896534062212763393.jpg',
 'jordanhefler-1896534106504538166.jpg']


# Load each feature vector from the saved .npy files
for image_name in images:   #feature_vectors.keys():
    load_path = os.path.join(feature_vector_dir, f"{image_name}.npy")
    if os.path.exists(load_path):
        loaded_feature_vectors[image_name] = np.load(load_path)
        print(f"Feature vector loaded for {image_name}")
    else:
        print(f"Feature vector file not found for {image_name}")

# Confirm loaded vectors
print("Loaded feature vectors:")
for image_name, vector in loaded_feature_vectors.items():
    print(f"{image_name}: {vector.shape}")


Feature vector loaded for jordanhefler-1896533999079959560.jpg
Feature vector loaded for jordanhefler-1896534007569376702.jpg
Feature vector loaded for jordanhefler-1896534017811791152.jpg
Feature vector loaded for jordanhefler-1896534029799223578.jpg
Feature vector loaded for jordanhefler-1896534034622533248.jpg
Feature vector loaded for jordanhefler-1896534043640325572.jpg
Feature vector loaded for jordanhefler-1896534051659720341.jpg
Feature vector loaded for jordanhefler-1896534060644024200.jpg
Feature vector loaded for jordanhefler-1896534062212763393.jpg
Feature vector loaded for jordanhefler-1896534106504538166.jpg
Loaded feature vectors:
jordanhefler-1896533999079959560.jpg: (768,)
jordanhefler-1896534007569376702.jpg: (768,)
jordanhefler-1896534017811791152.jpg: (768,)
jordanhefler-1896534029799223578.jpg: (768,)
jordanhefler-1896534034622533248.jpg: (768,)
jordanhefler-1896534043640325572.jpg: (768,)
jordanhefler-1896534051659720341.jpg: (768,)
jordanhefler-189653406064402420

In [9]:
from scipy.spatial.distance import cosine
import numpy as np
import pandas as pd
import torch.nn.functional as F  # Ensure F is imported from torch.nn.functional, not torchvision.transforms.functional

# List of image names for reference
image_names = list(loaded_feature_vectors.keys())


# List to store pairwise similarities
pairwise_similarities = []

# Calculate pairwise cosine similarity for each unique pair of images
for i, img1 in enumerate(image_names):
    for j in range(i + 1, len(image_names)):  # Only calculate for j > i to avoid duplicate pairs
        img2 = image_names[j]
        # Cosine similarity: 1 - cosine distance
        similarity = 1 - cosine(loaded_feature_vectors[img1], loaded_feature_vectors[img2])
        # Compute cosine similarity directly
        # Define two sample vectors as PyTorch tensors
        vector_a = torch.tensor(loaded_feature_vectors[img1], dtype=torch.float32)
        vector_b = torch.tensor(loaded_feature_vectors[img2], dtype=torch.float32)

        cos_similarity = F.cosine_similarity(vector_a.unsqueeze(0), vector_b.unsqueeze(0))
        #print(f"Cosine Similarity (from PyTorch): {cos_similarity.item()}")
        print(f"Cosine Similarity (from PyTorch):{img1} - {img2}: {cos_similarity.item():.4f}")

        pairwise_similarities.append((img1, img2, similarity))

# Sort the list of pairs by similarity in descending order
pairwise_similarities.sort(key=lambda x: x[2], reverse=True)

# Display sorted pairwise similarities
print("Pairwise Cosine Similarities (sorted):")
for img1, img2, similarity in pairwise_similarities:
    print(f"{img1} - {img2}: {similarity:.4f}")



# Initialize a list to store all pairwise similarity values (excluding self-similarities)
similarities = []

# Calculate pairwise cosine similarity for each unique pair of images
for i, img1 in enumerate(image_names):
    for j in range(i + 1, len(image_names)):  # Only calculate for j > i to avoid self-similarities
        img2 = image_names[j]
        # Cosine similarity: 1 - cosine distance
        similarity = 1 - cosine(loaded_feature_vectors[img1], loaded_feature_vectors[img2])
        similarities.append(similarity)

# Calculate the average similarity
average_similarity = sum(similarities) / len(similarities)

print(f"Average Cosine Similarity (excluding self-similarity): {average_similarity:.4f}")


Cosine Similarity (from PyTorch):jordanhefler-1896533999079959560.jpg - jordanhefler-1896534007569376702.jpg: 0.0054
Cosine Similarity (from PyTorch):jordanhefler-1896533999079959560.jpg - jordanhefler-1896534017811791152.jpg: 0.3790
Cosine Similarity (from PyTorch):jordanhefler-1896533999079959560.jpg - jordanhefler-1896534029799223578.jpg: 0.1177
Cosine Similarity (from PyTorch):jordanhefler-1896533999079959560.jpg - jordanhefler-1896534034622533248.jpg: 0.6043
Cosine Similarity (from PyTorch):jordanhefler-1896533999079959560.jpg - jordanhefler-1896534043640325572.jpg: 0.0300
Cosine Similarity (from PyTorch):jordanhefler-1896533999079959560.jpg - jordanhefler-1896534051659720341.jpg: -0.0119
Cosine Similarity (from PyTorch):jordanhefler-1896533999079959560.jpg - jordanhefler-1896534060644024200.jpg: 0.0891
Cosine Similarity (from PyTorch):jordanhefler-1896533999079959560.jpg - jordanhefler-1896534062212763393.jpg: 0.0516
Cosine Similarity (from PyTorch):jordanhefler-18965339990799595

In [12]:
import os

# Define the directory where feature maps are stored
feature_map_dir = 'saved_feature_maps'

# List all files in the directory
all_files = os.listdir(feature_map_dir)

# Filter to show only .npy files, assuming feature maps are saved with this extension
npy_files = [file for file in all_files if file.endswith('.npy')]

# Print some file names (e.g., first 5 files)
print("Some feature map files in the directory:")
for file_name in npy_files[:5]:  # Change the number to display more or fewer
    print(file_name)


Some feature map files in the directory:
katiejanehughes-1790061909840989942.jpg.npy
aforeverlylove-1719994352468268168.jpg.npy
mrsmajorhoff-1082870017949215842.jpg.npy
fitness.motivation588-1750828735300336274.jpg.npy
millicakrstic-1401471187862995970.jpg.npy


In [22]:
import os
import numpy as np

# Define the directory where feature maps are stored
feature_map_dir = 'saved_feature_maps_VGG19_4096size'#'saved_feature_maps' 

# List of image names without extensions
image_names = [
    'jordanhefler-1896533999079959560',
    'jordanhefler-1896534007569376702',
    'jordanhefler-1896534017811791152',
    'jordanhefler-1896534029799223578',
    'jordanhefler-1896534034622533248',
    'jordanhefler-1896534043640325572',
    'jordanhefler-1896534051659720341',
    'jordanhefler-1896534060644024200',
    'jordanhefler-1896534062212763393',
    'jordanhefler-1896534106504538166'
]

#image_names = ['fredbaierjr-1540631541485798570', 'fredbaierjr-1540631982348963186']

# Placeholder feature map in case the file does not exist
placeholder_feature_map = np.zeros((2048,), dtype=np.float32)  # Adjust shape if needed

# Dictionary to store loaded feature maps
feature_maps = {}



# Load feature maps
for img_name in image_names:
    feature_map_path = os.path.join(feature_map_dir, img_name + '.jpg' + '.npy')
    if os.path.exists(feature_map_path):
        feature_map = np.load(feature_map_path).astype(np.float32)
        print(f"Loaded feature map for {img_name}")
    else:
        feature_map = placeholder_feature_map
        print(f"Feature map for {img_name} not found. Using placeholder.")
    
    feature_maps[img_name] = feature_map

# Display summary
print(f"\nLoaded feature maps for {len(feature_maps)} images.")


Loaded feature map for jordanhefler-1896533999079959560
Loaded feature map for jordanhefler-1896534007569376702
Loaded feature map for jordanhefler-1896534017811791152
Loaded feature map for jordanhefler-1896534029799223578
Loaded feature map for jordanhefler-1896534034622533248
Loaded feature map for jordanhefler-1896534043640325572
Loaded feature map for jordanhefler-1896534051659720341
Loaded feature map for jordanhefler-1896534060644024200
Loaded feature map for jordanhefler-1896534062212763393
Loaded feature map for jordanhefler-1896534106504538166

Loaded feature maps for 10 images.


In [35]:
image_names = ['fredbaierjr-1540631541485798570,jpg', 'fredbaierjr-1540631982348963186.jpg']
low_lvl_feature_dir = 'low_lvl_image_features1'

img_name = image_names[1].strip()
            
# Load low-level features
low_level_features = {}
#low_level_dir = os.path.join(self.feature_dir, 'low_lvl_image_features')
for key in ['color_hist', 'contrast', 'lbp', 'brightness', 'background_simplicity', 'colorfulness', 'color_entropy']:
    feature_path = os.path.join(low_lvl_feature_dir, f"{img_name}_{key}.npy")
    if os.path.exists(feature_path):
        low_level_features[key] = np.load(feature_path).astype(np.float32)

low_level_features  

{'color_hist': array([9.27715469e-03, 1.16415322e-10, 1.16415322e-10, 7.73096341e-04,
        7.73096341e-04, 2.31928891e-03, 5.41167380e-03, 6.57131802e-03,
        1.43022798e-02, 2.55121756e-02, 3.51758786e-02, 5.48898317e-02,
        8.07885528e-02, 1.40316963e-01, 3.04213375e-01, 3.49439502e-01,
        5.72477758e-01, 8.46540391e-01, 9.99999940e-01, 9.16119039e-01,
        7.29802847e-01, 6.38964057e-01, 4.80865866e-01, 4.44143772e-01,
        3.96598369e-01, 3.63741785e-01, 3.20834935e-01, 2.81793565e-01,
        2.42752209e-01, 2.79087722e-01, 2.91070729e-01, 4.00077283e-01,
        3.02280635e-01, 3.67220700e-01, 2.98415154e-01, 3.06919187e-01,
        3.06146115e-01, 2.77928084e-01, 2.57054508e-01, 2.79860824e-01,
        2.17240036e-01, 2.46617690e-01, 2.43911862e-01, 2.31928870e-01,
        2.19945878e-01, 2.43138760e-01, 2.31928870e-01, 2.34634697e-01,
        2.46617690e-01, 2.58600682e-01, 2.32315421e-01, 2.72516429e-01,
        2.21878618e-01, 2.30382681e-01, 1.74719751

In [37]:
import os

# Define the directory
save_dir = 'saved_feature_vectors_ResNet50'

# Check if directory exists
if os.path.exists(save_dir):
    # Count the number of files in the directory
    num_files = len([file for file in os.listdir(save_dir) if os.path.isfile(os.path.join(save_dir, file))])
    print(f"Number of files in the directory: {num_files}")
else:
    print(f"Directory '{save_dir}' does not exist.")


Number of files in the directory: 1680649


In [7]:
import os

# Define the directory
save_dir = 'saved_feature_vectors_VisTrasformer_2'

# Check if directory exists
if os.path.exists(save_dir):
    # Count the number of files in the directory
    num_files = len([file for file in os.listdir(save_dir) if os.path.isfile(os.path.join(save_dir, file))])
    print(f"Number of files in the directory: {num_files}")
else:
    print(f"Directory '{save_dir}' does not exist.")

Number of files in the directory: 1680649


In [10]:
import numpy as np
import os

# Define the directory
save_dir = 'saved_feature_vectors_VisTrasformer_2'

# List all files in the directory with .npy extension
files = [file for file in os.listdir(save_dir) if file.endswith('.npy')]

# Check if there are numpy files in the directory
if files:
    # Take the first file
    first_file_path = os.path.join(save_dir, files[0])
    
    try:
        # Load the numpy file
        data = np.load(first_file_path)
        
        # Print the shape of the loaded data
        print(f"Shape of the file '{files[0]}': {data.shape}")
    except Exception as e:
        print(f"Error loading file: {e}")
else:
    print("No numpy files found in the directory.")


Shape of the file 'katiejanehughes-1790061909840989942.jpg.npy': (768,)


In [ ]:
def save_feature_maps(feature_maps, img_names, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    for feature_map, img_name in zip(feature_maps, img_names):
        save_path = os.path.join(save_dir, img_name + '.npy')
        np.save(save_path, feature_map.cpu().numpy())

def main():
    # Configuration
    image_dir = 'images_output/output_images'
    batch_size = 64
    save_dir = 'saved_feature_maps_Vis_transformer' #'saved_feature_maps'

    visual_input_size = 2048  # Output size of ResNet50

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    image_model = ImageFeatureExtractor_ViT().to(device)


    image_model.eval()

    for images, img_names in dataloader:
        images = images.to(device)
        with torch.no_grad():
            feature_maps = image_model(images)
        save_feature_maps(feature_maps, img_names, save_dir)

        # Print the shape of the feature vector for the first image in the batch
        #print(f"Feature map shape for the first image: {feature_maps[0].shape}")
        #break

        
        
    print(f"Process complete.")

if __name__ == '__main__':
    main()